In [1]:
from helper import *
from datasetcreator import *
from dependencies import *
from loaddata import review_data,business_data, rest_data, user_data, elite_udata, non_elite_udata

Using TensorFlow backend.


User Data Loaded
Businesses Data Loaded
Review Data Loaded


In [38]:
bid_2_stars_e = review_data.loc[review_data['iselite'] == True].groupby('business_id')['stars'].apply(list).to_dict()
bids = []
for bid, stars in bid_2_stars_e.items():
    if(len(stars) >= 40):
        bids.append(bid)
bids = set(bids)
print(len(bids))

In [22]:
col_list = ['user_id','review_count', 'useful', 'funny', 'cool', 'fans', 'average_stars', 'compliment_hot','compliment_more', 'compliment_profile', 'compliment_cute','compliment_list', 'compliment_note', 'compliment_plain','compliment_cool', 'compliment_funny', 'compliment_writer','compliment_photos']
udatacontrols = user_data[col_list]
uid2control = {x:y for x,y in zip(udatacontrols.values[:,0], udatacontrols.values[:,1:])}

In [63]:
def give_data(review_data, uid2control, requiredstar, bids):
    x = []
    y = []
    for i,review in review_data.iterrows():
        star = review['stars']
        uid = review['user_id']
        controls = list(uid2control[uid])
        if review['iselite']:
            controls.append(1)
        else:
            controls.append(0)
        if star == requiredstar:
            y.append(1)
        else:
            y.append(0)
        x.append(controls)
    x = np.array(x)
    y = np.array(y)
    print(x.shape)
    print(y.shape)
    return x,y

In [59]:
idx = [x in bids for x in review_data['business_id']]
relevantreviews = review_data[idx]

In [104]:
#x,y = give_data(relevantreviews, uid2control, 5, bids)
import pickle
x,y = pickle.load( open( "2star.p", "rb" ) )
print(x.shape)
print(y.shape)

(2455301, 18)
(2455301,)


In [105]:
import statsmodels.api as sm
logit = sm.OLS(y, x)
result = logit.fit()
print(result.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.067
Model:                            OLS   Adj. R-squared (uncentered):              0.067
Method:                 Least Squares   F-statistic:                          1.036e+04
Date:                Sat, 11 Apr 2020   Prob (F-statistic):                        0.00
Time:                        02:58:11   Log-Likelihood:                     -3.5449e+05
No. Observations:             2455301   AIC:                                  7.090e+05
Df Residuals:                 2455284   BIC:                                  7.092e+05
Df Model:                          17                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [78]:
mgef  = result.get_margeff('mean')
print(mgef.summary())

        Logit Marginal Effects       
Dep. Variable:                      y
Method:                          dydx
At:                              mean
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1         -6.199e-06        nan        nan        nan         nan         nan
x2          1.819e-05      0.595   3.06e-05      1.000      -1.165       1.165
x3          1.137e-05      0.334    3.4e-05      1.000      -0.654       0.654
x4          -3.17e-05      1.051  -3.02e-05      1.000      -2.061       2.061
x5         -2.083e-05      4.663  -4.47e-06      1.000      -9.139       9.139
x6            -0.0459        nan        nan        nan         nan         nan
x7          -9.69e-07        nan        nan        nan         nan         nan
x8             0.0001     10.046   1.05e-05      1.000     -19.690      19.690
x9         -3.827e-05      6.580  -5.82e-06      1.000    